In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as f

In [12]:
conv = torch.nn.Conv2d(1024, 1024, 3, )

In [13]:
conv.weight = 

torch.Size([1024, 1024, 3, 3])

In [9]:
torch.randn(7, 3, 3).shape 7 * 64 * 16

torch.Size([7, 3, 3])

In [11]:
1024 * 1024 * 7

7340032

In [ ]:


(B, 1024, w, h) -> (B, 1024, w * h) -> (B, 1024, C' * K) -> (B, 1024, C', K) = a

z(K, 10) -> h(K, 64) -> basis (K, L * L) -----

    def _get_a(self, x):
        
        batch, in_features, w, h = x.size()
        
        # (B, C, w, h) -> (B, C, w * h)
        x = x.reshape(batch, self._in_channels, -1) 
        # (B, C, w * h) -> (B, C, C' * K)
        if self._get_a_network is None:
            w = torch.empty(3, 5)
            nn.init.uniform_(w)
            self._get_a = nn.Parameter()
            self._get_a_network = nn.Sequential(
            nn.Linear(w * h, self._out_channels * self._basis_dim),
            nn.ReLU()
        )
        x = self._get_a_network(x) 
        # (B, C, C' * K) -> (B, C, C', K)
        return x.reshape(batch, self._in_channels, self._out_channels, self._basis_dim) 
      

In [ ]:
nn.Sequential()

In [126]:
class BasisConv2d(nn.Module):
    def __init__(
        self, 
        in_channels: int,
        out_channels: int,
        kernel_size: int, 
        stride: int = 1, 
        padding: int = 0, 
        dilation: int = 1,
        basis_dim: int = 7,
        hidden_dim: int = 10
    ):
        super().__init__()
        
        self._stride = stride
        self._padding = padding
        self._dilation = dilation
        self._in_channels = in_channels
        self._out_channels = out_channels
        self._kernel_size = kernel_size
        self._basis_dim = basis_dim
        self._hidden_dim = hidden_dim
        
        self._basis_generator = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, kernel_size * kernel_size),
        )
        weights_a = torch.empty(out_channels, in_channels, basis_dim)
        self._a = nn.Parameter(nn.init.uniform_(weights_a), requires_grad=True)
    
    def forward(self, x):
        weights = self._get_weights()
        return f.conv2d(x, 
                        weights, stride=self._stride, 
                        dilation=self._dilation, 
                        padding=self._dilation)
    
    def _get_weights(self):
        z = torch.randn(self._basis_dim, self._hidden_dim)
        basis = self._basis_generator(z)
        weights = self._a @ basis
        return weights.view(self._out_channels, self._in_channels, 
                               self._kernel_size, self._kernel_size)

In [127]:
pic = torch.randn(20, 16, 64, 64)

In [128]:
bconv = BasisConv2d(16, 32, kernel_size=3)
bconv(pic).shape

torch.Size([20, 32, 64, 64])

In [115]:
pic = torch.randn(20, 15, 64, 64)

In [116]:
bconv

BasisConv2d(
  (_basis_generator): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=9, bias=True)
  )
)

In [30]:
help(f.conv2d)

Help on built-in function conv2d in module torch:

conv2d(...)
    conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) -> Tensor
    
    Applies a 2D convolution over an input image composed of several input
    planes.
    
    This operator supports :ref:`TensorFloat32<tf32_on_ampere>`.
    
    See :class:`~torch.nn.Conv2d` for details and output shape.
    
    Note:
        In some circumstances when given tensors on a CUDA device and using CuDNN, this operator may select a nondeterministic algorithm to increase performance. If this is undesirable, you can try to make the operation deterministic (potentially at a performance cost) by setting ``torch.backends.cudnn.deterministic = True``. See :doc:`/notes/randomness` for more information.
    
    Note:
        This operator supports complex data types i.e. ``complex32, complex64, complex128``.
    
    
    Args:
        input: input tensor of shape :math:`(\text{minibatch} , \text{in\_channels} , iH , iW)